In [53]:
import pandas as pd
import numpy as np
import json
import ast

In [159]:
def clean_yelp(df1,df2):
    if df2:
        final = pd.concat([df1,df2])
    else:
        final = df1
    final = final.drop_duplicates(subset = "id",keep = "first").reset_index(drop = True)
    my_list = []
    for entry in range(len(final['categories'])):
        x = ast.literal_eval(final['categories'][entry])
        x = x[0]['alias']
        my_list.append(x)
    final["Restaurant Category 1"] = pd.Series(my_list)
    pickup_list = []
    delivery_list = []
    reserve_list = []
    for entry in range(len(final['transactions'])):
        y = ast.literal_eval(final['transactions'][entry])
        if "pickup" in y:
            pickup_list.append(1)
        else:
            pickup_list.append(0)
        if "delivery" in y:
            delivery_list.append(1)
        else:
            delivery_list.append(0)
        if "restaurant_reservation" in y:
            reserve_list.append(1)
        else:
            reserve_list.append(0)
    final['Pickup Available'] = pd.Series(pickup_list)
    final['Delivery Available'] = pd.Series(delivery_list)
    final["Reservation Available"] = pd.Series(reserve_list)
    my_prices = []
    for entry in range(len(final['price'])):
        if final['price'][entry]=="$":
            my_prices.append("Low")
        elif final['price'][entry]=="$$":
            my_prices.append("Medium")
        elif final['price'][entry]=="$$$":
            my_prices.append("High")
        elif final['price'][entry]=="$$$$":
            my_prices.append("Very High")
        else:
            my_prices.append(np.nan)
    final['Price Level'] = pd.Series(my_prices)
    num_photos = []
    for entry in range(len(final['photos'])):
        x = ast.literal_eval(final['photos'][entry])
        x = len(x)
        num_photos.append(x)
    final['number of photos'] = pd.Series(num_photos)
    city = []
    state = []
    for entry in range(len(final['location'])):
        x= ast.literal_eval(final['location'][entry])
        y = x['city']
        z = x['state']
        city.append(y)
        state.append(z)
    final['City'] = pd.Series(city)
    final['State'] = pd.Series(state)
    start_time = []
    end_time = []
    for entry in range(len(final['hours'])):
        if type(final['hours'][entry]) == float:
            start_time.append(np.nan)
            end_time.append(np.nan)
        else:
            x = ast.literal_eval(final['hours'][entry])
            start_time.append(x[0]['open'][0]['start'])
            end_time.append(x[0]['open'][0]['end'])
    final['Opening Time'] = pd.Series(start_time)
    final['Closing Time'] = pd.Series(end_time)
    
    messaging = []
    for entry in range(len(final['messaging'])):
        if type(final['messaging'][entry])!=float:
            messaging.append(1)
        else:
            messaging.append(0)
    final["Has Messaging"] = pd.Series(messaging)
    keep = ['id','is_closed',
    'is_claimed',
    'review_count',
    'rating',
    'Restaurant Category',
    'Pickup Available',
    'Delivery Available',
    "Reservation Available",
    'Price Level',
    'number of photos',
    'City',
    'State',
    'Opening Time',
    'Closing Time',
    'Has Messaging']
    final = final[final.columns[final.columns.isin(keep)]]
    #final = final.dropna(how = 'any')
    #final['is_closed'] = np.where(final['is_closed']==False,0,1)

    return final

In [160]:
#frisco = pd.read_csv("Frisco_1km.csv")
big_frisco = pd.read_csv("All_Available_Frisco_Data_MoreINFO.csv")
#frisco2 = pd.read_csv("YelpData_Frisco.csv")
#big_df = clean_yelp(frisco,frisco2)
test = clean_yelp(big_frisco,df2 = None)
test

,id,is_claimed,is_closed,review_count,rating,Pickup Available,Delivery Available,Reservation Available,Price Level,number of photos,City,State,Opening Time,Closing Time,Has Messaging
0,9VqZZiaiyGH_6smpvUKaQQ,True,False,127,4.5,0,0,0,Medium,3,Aubrey,TX,0700,2100,0
1,dPaJzxW0foPByEXbPFkh_Q,True,False,141,4.0,1,1,0,Medium,3,Aubrey,TX,1100,2130,0
2,RhL6hXqiz_nMz06Xx7tBcg,True,False,8,4.5,1,1,0,NaN,3,East Aubrey,TX,1500,2100,0
3,yawhLqszfIp9q1aGNdLoig,True,False,23,4.5,1,1,0,Medium,3,Aubrey,TX,1100,1800,0
4,g_XBHNmwfPUOI4qE_bBgPQ,True,False,83,3.0,1,1,0,NaN,3,Aubrey,TX,1100,1430,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
890,Ns_3CA-0JfKKzXOAVYWOhQ,True,False,131,4.0,0,0,0,Medium,3,Plano,TX,1100,2200,0
891,2pM19ps82tA6GhsR7Wav-g,True,False,436,2.5,0,1,0,Medium,3,Plano,TX,1700,0100,0
892,azu29xbMwwXSggNxGK8NTA,True,False,109,3.5,1,1,0,Low,3,Plano,TX,1045,2200,0
893,ouCXOe0T3gLxC0gxLnaMsA,True,False,97,3.0,1,1,0,Low,3,Plano,TX,0700,2100,0


In [ ]:
#take column and transform to column of list
#ast.literal_eval(frisco['categories'].iloc[0])

#big_df['categories'] = big_df['categories'].apply(lambda x :ast.literal_eval(x))
#create another "apply" with function that returns all values where "alias" is the key
#condense list into string and then split/expand the list
#allows us to be agnostic about the number of aliases

In [161]:
test.to_csv("Cleaned Frisco.csv")